---
# Day 2: Introduction to Machine Learning in Python
---

## 1. Introduction <a id='l_overview'></a>

The goal of today's lecture is to present unsupervised Machine Learning. We will learn about the most typical machine learning problems, such as dimensionality reduction, and how to approach these using the Python programmming language. These are the important concepts that we will cover:

- [Machine Learning](#l_ml)
- [Data sets](#l_ds)
- [Dimensionality reduction](#l_dr) 
- [Principal Component Analysis (PCA)](#l_pca)
- [Multidimensional Scaling (MDS)](#l_mds)
- [Other dimensionality reduction techniques](#l_other)


## 2. Machine Learning <a id='l_ml'></a>

Below is the outline of the field with specific algorithms:

1. **Unsupervised Learning** - there is no correct input/output pair 
    - *Clustering*
        - K-Means
        - Hierarchical
        - Spectral
    - *Dimensionality reduction*
        - Principal Components Analysis (PCA)
        - Multidimensional Scaling (MDS)
        - Stochastic Neighbour Embedding (t-SNE)
        - Uniform Manifold Approximation and Projection (UMAP)
        
        
2. **Supervised Learning** - there is a correct input/output pair
    - *Regression*
        - Curve fitting
        - Linear regression 
    - *Classification*
        - Linear Classifiers (Support Vector Machines, Logistic regression)
        - Decision Trees
        - Neural Networks
        
        
3. **Reinforcement Learning** - is an area concerned with how software agents have to take actions in an environment so as to maximize some cumulative reward



## 3. Generating data sets

Setup:
- Suppose one has $p$ samples of N-dimensional data points, $x_i\in\mathbb{R}^N$
- Store these samples columnwise as $X\in\mathbb{R}^{p\,\times\,N}$
- We call this the original data matrix, or simply the data
- Assumption: there is a meaningful metric (e.g. Euclidean distance) on the data space (high dim)
- Assumption: there is a meaningful metric (e.g. Euclidean distance) on the latent space (low dim)

In [6]:
import numpy as np
import matplotlib.pyplot as plt

# plt.style.use('seaborn-darkgrid')
# plt.style.use('seaborn-poster')

## 3.1 Generate linear data with noise (2-dimensional data set) with 100 points:

In [7]:
raw_data_x = np.random.uniform(0,10, size=(100,))
raw_data_y = 0.5 * raw_data_x + np.random.normal(0,1,len(raw_data_x))

X_2d = np.empty((100, 2))
X_2d[:,0] = raw_data_x
X_2d[:,1] = raw_data_y

Visualize

In [23]:
plt.figure(dpi=250, figsize=(3,3))
plt.scatter(X_2d[:,0], X_2d[:,1], marker='.', color='steelblue', edgecolor='k', lw=0.5)
plt.xlabel(r'$X$')
plt.ylabel(r'$Y$')
plt.title('Two-dimensional dataset')
plt.tight_layout()
plt.show()

Let's look how the data looks like (first 10 points):

In [21]:
X_2d[:10]

## 3.2 Load high-dimensional data from the [MNIST](https://en.wikipedia.org/wiki/MNIST_database) dataset. 
![](pics/mnist.png)

Let's take only 1000 data points

In [22]:
import pandas as pd
df = pd.read_csv('data/mnist_test.csv')
df

We can now visualize what these data points represent (digital images)

In [20]:
# Plot the first 20 digits
fig, axes = plt.subplots(2, 10, figsize=(16, 6))
for i, j in enumerate(np.random.choice(np.arange(1000), size=20)):
    image = np.array(df.iloc[j, 1:]).reshape(28,28)
    label = np.array(df.iloc[j, 0])
    axes[i//10, i%10].imshow(image, cmap='gray');
    axes[i//10, i%10].axis('off')
    axes[i//10, i%10].set_title(f"digit: {label}")
    
plt.tight_layout()

### We will only use the first 1000 digits

In [9]:
X = np.array(df.iloc[:1000, 1:]).reshape(-1, 28, 28)
Y = np.array(df.iloc[:1000, 0])
print(Y.shape, X.shape)

(1000,) (1000, 28, 28)


In [19]:
plt.imshow(X[567], cmap='gray')
print(Y[567])
plt.show()

We want to convert each data point (picture with a handwritten digit) back to a vector which dimensionality is 28x28 = 784. i.e. to make each 28x28 matrix a flat vector

In [11]:
X = X.reshape(1000, 784)
X[567].shape

(784,)

## Summary - we have two data sets:
- 2-dimensional data set with 100 points
- 784-dimensional data set with 1000 points

## 4. Dimensionality reduction <a id='l_dr'></a>

Dimensionality reduction is a technique used in machine learning and data analysis to reduce the number of input features or variables of a dataset while still retaining the important information. This is done by projecting the high-dimensional data onto a lower-dimensional space, while preserving the relevant characteristics of the original data.

The main goal of dimensionality reduction is to simplify the dataset and make it more manageable for analysis, visualization, and modeling. It also helps to reduce the risk of overfitting and improve the performance of machine learning models by removing irrelevant or redundant features.

There are two main types of dimensionality reduction:

- Feature selection: In this method, a subset of the original features is selected based on some criteria, such as correlation or importance.

- Feature extraction: In this method, a new set of features is created by transforming the original features into a lower-dimensional space using techniques such as principal component analysis (PCA), singular value decomposition (SVD), or t-distributed stochastic neighbor embedding (t-SNE).

Overall, dimensionality reduction is a powerful tool for reducing the complexity of large datasets while still preserving the essential information needed for effective analysis and modeling. You can select a subset of original variables, or find a linear or nonlinear combination of features, or make a projection to lower dimensions. 

![](pics/dr.png)


Methods:
- **Principal Components Analysis (PCA)** - linear method to extract dimensions with the highest variance
- **Multidimensional Scaling (MDS)** - nonlinear method to project in lower dimensions by saving pairwise distances
- **Stochastic Neighbour Embedding (t-SNE)** - making an embedding in lower dimensions by conserving distribution of distances 
- **Uniform Manifold Approximation and Projection (UMAP)** - projecting the data on manifold into fewer dimensions

## 5. Principal Component Analysis <a id='l_dr'></a>

### **Math**:

- **PCA goal**: Find orthogonal transformation $W$ of the centered data $X_c$ (i.e. $Y=WX_c$) such that variance along subsequent components is maximized (i.e. most variance along first, the second most variance is along the second, etc.); 
- Note that $X_c$ is $p \times N$, $W$ is $N \times N$, $Y$ is $p \times N$, principal components are the columns of $W$.
- Principal components of $X_c$ are typically found via eigendecomposition of covariance matrix $X_c^T X_c$ .
- The PCA embedding is $Y=U^T X_c$, where $U$ stores columnwise eigenvectors of $X_c^T X_c$ in decreasing order (by eigenvalue).

### Compute principle components via eigenvectors of covariance matrix

1. Center data set, i.e. first subtract the mean of the dataset from the dataset.
2. Compute the covariance matrix $X_c^T X_c$.
3. Compute eigenvectors of $X_c^T X_c$ and order them in terms of decreasing eigenvalues.
4. Transform the data using the eigenvectors stored columnwise in a matrix $U$ by $Y=U^T X_c$.
5. Compare our step-by-step method to the pythonic library PCA implementation

## Now we apply PCA method to 2d dataset:

In [18]:
# Step 1

X_2d_centered = X_2d - np.mean(X_2d, axis=0)

# Visualize
fig, (a0, a1) = plt.subplots(1, 2, figsize=(16, 8), dpi=250)

a0.scatter(X_2d[:,0], X_2d[:,1], label='original data', color='steelblue', edgecolor='k', lw=0.5,)
a1.scatter(X_2d_centered[:,0], X_2d_centered[:,1], label='centered data', color='deepskyblue', edgecolor='k', lw=0.5,)
a0.legend()
a1.legend()

a0.set_xlabel(r'$X$')
a0.set_ylabel(r'$Y$')

a1.set_xlabel(r'$X$')
a1.set_ylabel(r'$Y$')

plt.show()

We see that the data is now centered on the origin.
Now compute the transformation:

In [13]:
# Step 2.
Cov = np.dot(np.transpose(X_2d_centered), X_2d_centered)
print("Covariance matrix:")
print(Cov)

Covariance matrix:
[[787.58308328 372.79326446]
 [372.79326446 275.21939021]]


In [17]:
#Step 3
eigvals, W = np.linalg.eig(Cov)
print("\nEigenvalues:")
print(eigvals)
print("\nEigenvectors (columns)")
print(W)

print("\nCheck that eigenvectors are orthogonal by computing their inner product (<w1,w2>=0):")
print(np.dot(W[:,0],W[:,1]))

print('\nVariance in the first principal component: {}'.format(eigvals[0]/np.sum(eigvals)))
print('Variance in the second principal component: {}'.format(eigvals[1]/np.sum(eigvals)))

Let's plot the eigenvectors in comparison to the data

In [16]:
plt.figure(dpi=250, figsize=(4,4))

plt.scatter(X_2d_centered[:,0], X_2d_centered[:,1], color='steelblue', edgecolor='k', lw=0.5, label='data points')
plt.xlabel(r'$X$')
plt.ylabel(r'$Y$')
plt.title('Two-dimensional dataset')

plt.plot([0, W[0][1]*3],[0, W[1][1]*3],'k',lw=4, label='eigenvector 2', ls='--')
plt.plot([0, W[0][0]*5],[0, W[1][0]*5],'k',lw=4, label='eigenvector 1')
plt.axis('equal')
plt.legend(fancybox=True, shadow=True)
plt.show()

Now we will apply the transformation to the data and plot the data in the new space. We flip the matrix W and corresponding eigenvalues so that they are ordered the same way as in the theory.

In [16]:
#Step 4

#eigvals = eigvals[::-1]
#print(eigvals)

# Applying transformation
X_2d_transformed = np.dot(X_2d_centered, W)

Visualize:

In [15]:
plt.figure(dpi=250, figsize=(4,4))
plt.scatter(X_2d_transformed[:,0], X_2d_transformed[:,1], color='tomato', edgecolor='k', lw=0.5, label='data points')
plt.title('Two-dimensional dataset transformed with manual PCA', fontsize=5)
plt.xticks(fontsize=5)
plt.yticks(fontsize=5)
plt.xlabel('PC 1', fontsize=5)
plt.ylabel('PC 2', fontsize=5)
plt.xlim(-8, 8)
plt.ylim(-4, 4)
plt.show()

Let's compare our naive implementation to the PCA implementation from sklearn

In [18]:
from sklearn.decomposition import PCA 

skl_PCA = PCA(n_components = 2).fit(X_2d) # fit the data to receive eigenvectors of covariance matrix
skl_X_2d_transformed = skl_PCA.transform(X_2d) # apply a transformation

Visualize and compare:

In [14]:
# Visualize
fig, (a0, a1) = plt.subplots(1, 2, figsize=(16, 4))

a0.scatter(skl_X_2d_transformed[:,0], skl_X_2d_transformed[:,1], label='sklearn PCA', color='steelblue', edgecolor='k',lw=0.5)
a1.scatter(X_2d_transformed[:,0], X_2d_transformed[:,1], label='our own PCA', color='tomato', edgecolor='k',lw=0.5)
a0.legend()
a1.legend()

a0.set_xlabel(r'$PC 1$')
a0.set_ylabel(r'$PC 2$')

a1.set_xlabel(r'$PC 1$')
a1.set_ylabel(r'$PC 2$')

a0.set_xlim(-8, 8)
a0.set_ylim(-4, 4)

a1.set_xlim(-8, 8)
a1.set_ylim(-4, 4)

plt.show()

Looks (almost) identical, up to 180 degree rotation! 

We will now truncate the data to one dimension and see how it looks. It's called a simple PCA dimensionality reduction.

In [12]:
print(skl_PCA.explained_variance_ratio_)

We notice that >90% of variance is in the first principal component

In [11]:
plt.scatter(skl_X_2d_transformed[:,0] , np.zeros(shape=skl_X_2d_transformed[:,0].shape), label='sklearn PCA', color='lime', edgecolor='w', lw=2, marker='o')
plt.ylim((-6,6))
plt.xlim((-8,8))
plt.legend()
plt.yticks([])
plt.xlabel('PC 1')

plt.show()

## Exercise 5.1

Perform principal component analysis on the 1000 points of 784-dimensional MNIST dataset using `sklearn`.
The dataset is already in the memory of the Jupyter Notebook under variable `X`. `Y` contains the label of each handwritten digit, i.e. the number, or the class. [Documentation on sklearn PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) 

784-dimensional dataset has 784 principal components (PC). 

1. Plot the percent variance contained in each PC vs PC number. **Hint**: variable `explained_variance_ratio_` may be useful.
2. Now plot cumulative percent variance vs number of PC components used. Decide how many PC you need to capture 90% of total variance.
2. Use the first two principal components to represent MNIST data set in two dimensions on a scatter plot. Each mnist digit will now be represented as a point. 
3. Show the image of first two principal eigenvectors, rescaled as 28x28. **Hint**: variable `components_` and function `reshape()` might be useful.


## 6. Multidimensional Scaling (MDS) <a id='l_mds'></a>

#### Metric MDS
Setup:
- Suppose one has $p$ samples of N-dimensional data points, $x_i\in\mathbb{R}^N$
- Store these samples rowwise as $X\in\mathbb{R}^{p\,\times\,N}$
- We call this the original data matrix, or simply the data
- Assumption: there is a meaningful metric (e.g. Euclidean distance) on the data space (high dim)
- Assumption: there is a meaningful metric (e.g. Euclidean distance) on the latent space (low dim)

Goal:
- Given N-dim data $X$, a metric $d(\cdot,\cdot)$ on $\mathbb{R}^N$, a target dimension $k<N$, and a metric $g(\cdot,\cdot)$ on $\mathbb{R}^k$
- Find an embedding $Y\in\mathbb{R}^{k\,\times\,p}$ (i.e. a $y_i\in\mathbb{R}^k$ for each $x_i\in\mathbb{R}^N$) such that distances $d_{ij}$, $g_{ij}$ are preserved between representations

Objective function: $$Y^\ast=\operatorname*{arg\,min}_Y {\sum_{i<j}{\left|d_{ij}\left(X\right)-g_{ij}\left(Y\right)\right|}}$$

Let's not invent the wheel and use MDS implementation in Python:

In [22]:
from sklearn.manifold import MDS
from scipy.spatial.distance import pdist, squareform
# compute MDS embedding (2D)
# Docs: http://scikit-learn.org/stable/modules/generated/sklearn.manifold.MDS.html

### By knowing the pairwise distances between points, can we guess their coordinates (without knowing them explicitly )?

In [9]:
## First, we should find pairwise distances between points in our 2D dataset:

D_compressed = pdist(X_2d, metric='euclidean') # n*(n-1)/2 size
D = squareform(D_compressed)

In [10]:
D.shape

The pairwise distance matrix looks something like this: 

![](https://www.displayr.com/wp-content/uploads/2018/04/Distance-Matrix.png)

Let's run the MDS algorithm on the pairwise distance matrix D:

In [25]:
mds_2d = MDS(n_components=2, dissimilarity='precomputed', n_jobs=-1 ).fit_transform(D) 
# it contains the embedding of original 2D data back on 2D

### Now let's see the output of the algorithm.

In [8]:

fig, (a0, a1) = plt.subplots(1, 2, figsize=(16, 8))

a0.scatter(X_2d[:,0], X_2d[:,1], label='original data', color='steelblue', edgecolor='k', lw=0.5)
a0.set_xlabel(r'$X$')
a0.set_ylabel(r'$Y$')

a1.scatter(mds_2d[:,0], mds_2d[:,1], label='data reconstructed with MDS', color='deepskyblue', edgecolor='k', lw=0.5)
a1.set_xlabel('MDS axis 1')
a1.set_ylabel('MDS axis 2')

a0.legend()
a1.legend()

plt.show()

### Restores original coordinates up to a (i) translation, (ii) rotation and (iii) reflection.

What about high dimensional data?

In [27]:
mds_X = MDS(n_components=2, dissimilarity='euclidean', n_jobs=4).fit_transform(X) ## distances will be computed

This is what we get as an output of the algorithm

In [28]:
mds_X.shape

(1000, 2)

In [6]:
plt.scatter(mds_X[:,0], mds_X[:,1], color='steelblue', edgecolor='k', lw=0.5)
plt.xlabel('MDS axis 1')
plt.ylabel('MDS axis 2')
plt.title('Representation of 784-dimesional space in 2D (with preserved distances)')
plt.show()

In [7]:
for label in set(Y):
    mask = Y==label
    plt.scatter(mds_X[:,0][mask], mds_X[:,1][mask], label = label, edgecolor='k', lw=0.5)
plt.legend()

plt.xlabel('MDS axis 1')
plt.ylabel('MDS axis 2')
plt.show()

## Exercise 6.1. 
Find the relative coordinates (sketch of the map) of the European cities knowing only pairwise distances between them.

Remember that MDS is a stochastic algorithm and may return different results at each run. Your embeddings may require addtional rotation and reflection to preserve geographical locations. To make your algorithm robust, apply rotations and reflections after each run of MDS. Use the information that Stockholm is directly North of Munich and Athens should be East of Lisbon. Put city labels on your scatter plot.

You should get relative locations similar to these:
![](pics/europe.jpg)

To load data, use the code below:

In [5]:
# import pandas as pd
# # Pairwise distance between European cities
# url = 'https://media.githubusercontent.com/media/neurospin/pystatsml/master/datasets/eurodist.csv'
# df = pd.read_csv(url)
# df

In [4]:
# Array with cities:
city = np.array(df["city"])
print(city)

# Squareform distance matrix is here
D = np.array(df.iloc[:, 1:]) 
print(D.shape)

In [ ]:
# Write your code here


## 7. Other dimensionality reduction techniques (non-linear) t-SNE, UMAP

In [36]:
from sklearn.manifold import TSNE

### [t-SNE](https://lvdmaaten.github.io/tsne/) 

In [37]:
%%time

tsne_2d = TSNE(n_components=2).fit_transform(X)

CPU times: total: 4.28 s
Wall time: 7.83 s


### Visualize

In [3]:
plt.scatter(tsne_2d[:,0], tsne_2d[:,1], edgecolor='k', lw=0.5, color='steelblue')
plt.title('2D t-SNE represetation of MNIST data set')
plt.xlabel('t-SNE axis 1')
plt.xlabel('t-SNE axis 2')
plt.show()

Show with labels:

In [2]:
for label in set(Y):
    mask = Y==label
    plt.scatter(tsne_2d[:,0][mask], tsne_2d[:,1][mask], label = label, edgecolor='k', lw=0.5) 
    
plt.title('2D t-SNE represetation of MNIST data set')
plt.xlabel('t-SNE axis 1')
plt.xlabel('t-SNE axis 2')
plt.legend()
plt.show()

### [UMAP](https://github.com/lmcinnes/umap) - is a very fast algorithm. My favorite

In [69]:
# conda install -c conda-forge umap-learn # download in conda venv, first.

In [40]:
%%time

import umap
umap_2d = umap.UMAP(n_components=2).fit_transform(X)

### Visualize

In [1]:
plt.figure(dpi=250, figsize=(4,4))
plt.scatter(umap_2d[:,0], umap_2d[:,1], edgecolor='k', lw=0.5, color='steelblue')
plt.title('2D UMAP represetation of MNIST data set')
plt.xlabel('UMAP axis 1')
plt.xlabel('UMAP axis 2')
plt.show()

Show with labels:

In [ ]:
for label in set(Y):
    mask = Y==label
    plt.scatter(umap_2d[:,0][mask], umap_2d[:,1][mask], marker='o', edgecolor='w', lw=2, label = label) 
plt.title('2D UMAP represetation of MNIST data set')
plt.xlabel('UMAP axis 1')
plt.xlabel('UMAP axis 2')
plt.legend()
plt.show()

## Data exploration: why some of the digits are clustered around other digits?

In [ ]:
mask = (umap_2d[:, 0] < 2) & (Y == 6) & (umap_2d[:, 0] > 1)
plt.imshow(X[mask].reshape(28, 28))